In [8]:
import sys
sys.path.append("./src")

from pygler.viewer import PyGLer
from pygler.model import PyGLerModel,Geometry
import numpy as np
import cv2

import sys
import tinyobjloader
from OpenGL.GL import *
from OpenGL.GLU import *
filename = "cube.obj"

In [1]:
reader = tinyobjloader.ObjReader()

# Load .obj(and .mtl) using default configuration
ret = reader.ParseFromFile(filename)

attrib = reader.GetAttrib()
print("attrib.vertices = ", len(attrib.vertices))
print("attrib.normals = ", len(attrib.normals))
print("attrib.texcoords = ", len(attrib.texcoords))

# vertex data must be `xyzxyzxyz...`
assert len(attrib.vertices) % 3 == 0

# normal data must be `xyzxyzxyz...`
assert len(attrib.normals) % 3 == 0

# texcoords data must be `uvuvuv...`
assert len(attrib.texcoords) % 2 == 0

for (i, v) in enumerate(attrib.vertices):
    print("v[{}] = {}".format(i, v))

for (i, v) in enumerate(attrib.normals):
    print("vn[{}] = {}".format(i, v))

for (i, v) in enumerate(attrib.texcoords):
    print("vt[{}] = {}".format(i, v))

print("numpy_vertices = {}".format(attrib.numpy_vertices()))

shapes = reader.GetShapes()
print("Num shapes: ", len(shapes))
for shape in shapes:
    print(shape.name)
    print("len(num_indices) = {}".format(len(shape.mesh.indices)))
    for (i, idx) in enumerate(shape.mesh.indices):
        print("[{}] v_idx {}".format(i, idx.vertex_index))
        print("[{}] vn_idx {}".format(i, idx.normal_index))
        print("[{}] vt_idx {}".format(i, idx.texcoord_index))
    print("numpy_indices = {}".format(shape.mesh.numpy_indices()))
    print("numpy_num_face_vertices = {}".format(shape.mesh.numpy_num_face_vertices()))
    print("numpy_material_ids = {}".format(shape.mesh.numpy_material_ids()))

In [2]:
windowWidth=1024
windowHeight=768
red,green,blue,alpha = 0.3,0.4,0.5,1.0
glClearColor(red,green,blue,alpha)

In [3]:
viewer = PyGLer(useFBO=True)
viewer.start()

In [4]:
tri = PyGLerModel.LoadObj("cube.obj",computeNormals=True)
viewer.addModel(tri)

In [5]:
depth,bgr = viewer.Convert2BGRD(viewer.capture())

In [6]:
k=0
while k&255!=ord('q'):
    depth,bgr = viewer.Convert2BGRD(viewer.capture())
    depth = cv2.normalize(depth,None,0,255,cv2.NORM_MINMAX,cv2.CV_8UC1)
    cv2.imshow("depth",depth)
    k = cv2.waitKey(1)

cv2.destroyAllWindows()


In [7]:
s = 0.5                                                                
v = np.array([ s, s, s,1,
              -s, s, s,1,
              -s,-s, s,1,
               s,-s, s,1,
               s,-s,-s,1,
               s, s,-s,1,
              -s, s,-s,1,
              -s,-s,-s,1 ],dtype='f').reshape(-1,4)
              
c = np.array([ 1, 1, 1, 1, 
               1, 1, 0, 1, 
               1, 0, 1, 1,
               0, 1, 1, 1,
               1, 0, 0, 1,
               0, 0, 1, 1,
               0, 1, 0, 1,
               0, 0, 0, 1],dtype='f').reshape(-1,4); 

t = np.array([0,1,2, 2,3,0,
              0,3,5, 5,3,4,
              5,6,7, 7,4,5,
              1,0,5, 5,6,1,
              1,6,7, 7,2,1,
              2,7,4, 4,3,2 ],dtype=np.uint32)
                                 
vertices = np.array( [
        [  0, 1, 0, 1.0 ],
        [ -1,-1, 0, 1.0 ],
        [  1,-1, 0, 1.0 ],
        [  2,-1, 0, 1.0 ],
        [  4,-1, 0, 1.0 ],
        [  4, 1, 0, 1.0 ],
        [  2,-1, 0, 1.0 ],
        [  4, 1, 0, 1.0 ],
        [  2, 1, 0, 1.0 ]
    ],'f')

triangles = np.array([0,1,2,3,4,5,6,7,8]
,np.uint32)

colors = np.array([ [ 0,1,0,1 ],
                   [ 1,1,0,1 ],
                   [ 0,1,1,1 ],
                   [ 1,0,0,1 ],
                   [ 0,1,0,1 ],
                   [ 0,0,1,1 ],
                   [ 1,0,0,1 ],
                   [ 0,0,1,1 ],
                   [ 0,1,1,1 ]],'f')


In [ ]:
m = PyGLerModel("Test",Geometry(vertices,triangles=triangles, colors=colors))

In [ ]:
cube = PyGLerModel("Cube",Geometry(vertices = v, triangles=t, colors=c, autoScale=True))

In [ ]:
viewer.addModel(cube)